In [7]:
from openai import OpenAI #############Prompt 1 SID

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "mistral-small"  # model used

client = OpenAI(api_key=api_key, base_url=base_url)


# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
But if the item is not in stock, order is declined.
Customer is sent order declined message.
Customer then decides upon further action either the customer decides to cancel order and it ends
or customer chooses another e-commerce supplier.
"""


# Create the prompt
prompt = f"""
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Given the following scenario:

\"\"\"{scenario}\"\"\"

Your task is to generate an SID (Subject Interaction Diagram) from the scenario above:

### 1. Subject Interaction Diagram (SID):
- Identify all the **subjects** involved.
- Describe their **interactions** using messages in **noun form only** (e.g., "Order", "Inventory Check Request", "Confirmation Email").
- Avoid using verbs like “place”, “send”, “check”, or “confirm”.
- Use noun phrases that represent the content of the interaction, not the action.
- Return the SID as a numbered list of interactions using the format:
  Subject A -> Subject B: Message (noun form)
  
  
### Use the following format (example based on a healthcare scenario):

### Example Scenario:
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### Subjects:
- Patient
- Healthcare App
- Hospital System

### SID:
1. Patient -> Healthcare App: Appointment Request  
2. Healthcare App -> Hospital System: Appointment Request  
3. Hospital System -> Healthcare App: Appointment Confirmation  
4. Healthcare App -> Patient: Appointment Confirmation
5. Hospital System -> Healthcare App: Appointment Declination
6. Healthcare App -> Patient: Appointment Declination
7. Patient -> Healthcare App: Appointment Request (another hospital)
8. Patient -> Healthcare App: Appointment Cancellation Request

Now, do the same for the scenario provided above.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5
)

# Output
print(completion.choices[0].message.content)

### Subjects:
- Customer
- E-commerce Website
- Inventory System
- Payment Gateway
- Order Confirmation System

### SID:
1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Payment Request
5. Customer -> Payment Gateway: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Order Confirmation System: Order Confirmation Request
8. Order Confirmation System -> Customer: Order Confirmation Email
9. Inventory System -> E-commerce Website: Out of Stock Notification
10. E-commerce Website -> Customer: Order Declination Message
11. Customer -> E-commerce Website: Order Cancellation Request
12. Customer -> Another E-commerce Supplier: Order

This list captures the interactions between the subjects involved in the e-commerce order process, ensuring that each message is in noun form and avoiding ve

In [3]:
# Get the actual response from Mistral. Now, use the saved Mistral responses in variables sid_response_text and mistral_output below
#from Prompt 2 SBD and Prompt 1 SID
#mistral_output = completion.choices[0].message.content

import re

sid_response_text = """
### SID:
1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Payment Request
5. Customer -> Payment Gateway: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Order Confirmation System: Order Confirmation Request
8. Order Confirmation System -> Customer: Order Confirmation Email
9. Inventory System -> E-commerce Website: Out of Stock Notification
10. E-commerce Website -> Customer: Order Declination Message
11. Customer -> E-commerce Website: Order Cancellation Request
12. Customer -> Another E-commerce Supplier: Order
"""


mistral_output = """
### SBD for the Customer:

1. **StartState**: Decide to place order
2. **SendState**: Send Order to E-commerce Website
3. **ReceiveState**: Receive Response from E-commerce Website
   Choices:
     - From: E-commerce Website
       Msg: Payment Request
       Next: 4
     - From: E-commerce Website
       Msg: Order Declination Message
       Next: 5

4. **SendState**: Send Payment to Payment Gateway
5. **ReceiveState**: Receive Reply from Payment Gateway
   Choices:
     - From: Payment Gateway
       Msg: Payment Confirmation
       Next: 6
     - From: Payment Gateway
       Msg: Payment Declination
       Next: 7

6. **EndState**: Order Confirmed
7. **DoState**: Decide upon further action
   Branches:
     - Step: 8
       Description: Cancel order
     - Step: 10
       Description: Choose another e-commerce supplier

8. **SendState**: Send Order Cancellation Request to E-commerce Website
9. **EndState**: Order Cancelled
10. **GotoStep**: 2  # Retry order loop

### Explanation:
- **Step 1**: The customer decides to place an order.
- **Step 2**: The customer sends the order to the e-commerce website.
- **Step 3**: The customer receives a response from the e-commerce website. Depending on the response, the flow branches:
  - If the customer receives a `Payment Request`, the customer proceeds to pay.
  - If the customer receives an `Order Declination Message`, the customer decides upon further action.
- **Step 4**: The customer sends the payment to the payment gateway.
- **Step 5**: The customer receives a response from the payment gateway. Depending on the response, the flow branches:
  - If the customer receives a `Payment Confirmation`, the order is confirmed.
  - If the customer receives a `Payment Declination`, the customer decides upon further action.
- **Step 6**: The order is confirmed and the process ends.
- **Step 7**: The customer decides upon further action if the payment is declined. The customer can choose to cancel the order or choose another supplier.
- **Step 8**: The customer sends an order cancellation request to the e-commerce website and the process ends.
- **Step 10**: If the customer chooses to try another supplier, the process loops back to the original order step by using `GotoStep: 2`.
"""


sbd_response_text ="""

### SBD for Customer:

1. StartState: Decide to place order
2. SendState: Send Order to E-commerce Website
3. ReceiveState: Receive Reply from E-commerce Website
   Choices:
   - From: E-commerce Website
     Msg: Payment Request
     Next: 4
   - From: E-commerce Website
     Msg: Order Declination Message
     Next: 7

4. SendState: Send Payment to Payment Gateway
5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
   - From: Payment Gateway
     Msg: Payment Confirmation
     Next: 6
   - From: Payment Gateway
     Msg: Payment Declination
     Next: 7

6. EndState: Order Confirmed and Paid
7. DoState: Decide upon further action
   Branches:
     - Step: 8
       Description: Cancel order
     - Step: 9
       Description: Choose another supplier

8. SendState: Send Order Cancellation Request to E-commerce Website
9. EndState: Order Cancelled

10. GotoStep: 2  # Retry order loop with another supplier
"""

sbd_response_text1 ="""

### SBD for Customer:


1. StartState: Decide to place an order
2. SendState: Send Order to E-commerce Website
3. ReceiveState: Receive Reply from E-commerce Website
   Choices:
   - From: E-commerce Website
     Msg: Payment Request
     Next: 4
   - From: E-commerce Website
     Msg: Order Declination Message
     Next: 6

4. SendState: Send Payment to Payment Gateway
5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
   - From: Payment Gateway
     Msg: Payment Confirmation
     Next: 7
   - From: Payment Gateway
     Msg: Payment Declination
     Next: 8

6. DoState: Decide upon further action
   Branches:
     - Step: 9
       Description: Cancel order
     - Step: 10
       Description: Choose another supplier

7. EndState: Order Confirmed and Payment Successful

8. DoState: Decide upon further action
   Branches:
     - Step: 9
       Description: Cancel order
     - Step: 10
       Description: Choose another supplier
9. SendState: Send Order Cancellation Request to E-commerce Website
10. EndState: Order Cancelled

10. GotoStep: 2  # Retry order loop
"""

sbd_response_text2 = """
### SBD for Customer:

1. StartState: Decide to place order
2. SendState: Send Order to E-commerce Website
3. ReceiveState: Receive Reply from E-commerce Website
   Choices:
   - From: E-commerce Website
     Msg: Payment Request
     Next: 4
   - From: E-commerce Website
     Msg: Order Declination Message
     Next: 6

4. SendState: Send Payment to Payment Gateway
5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
   - From: Payment Gateway
     Msg: Payment Confirmation
     Next: 7
   - From: Payment Gateway
     Msg: Payment Declination
     Next: 8

6. DoState: Decide upon further action
   Branches:
     - Step: 9
       Description: Cancel order
     - Step: 11
       Description: Choose another supplier

7. EndState: Order Confirmed
8. DoState: Decide upon further action
   Branches:
     - Step: 9
       Description: Cancel order
     - Step: 11
       Description: Choose another supplier

9. SendState: Send Order Cancellation Request to E-commerce Website
10. EndState: Order Cancelled
11. GotoStep: 2  # Retry order loop
"""

###########Mistral Response Obtained with Send States To and Msg Added
sbd_response_text3 = """
### SBD:

#### Customer:
1. StartState: Decide to place order
2. SendState: Send Order to E-commerce Website
   To: E-commerce Website
   Msg: Order
3. ReceiveState: Receive Response from E-commerce Website
   Choices:
   - From: E-commerce Website
     Msg: Payment Request
     Next: 4
   - From: E-commerce Website
     Msg: Order Declination Message
     Next: 5
4. SendState: Send Payment to Payment Gateway
   To: Payment Gateway
   Msg: Payment
5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
   - From: Payment Gateway
     Msg: Payment Confirmation
     Next: 6
   - From: Payment Gateway
     Msg: Payment Declination
     Next: 7
6. EndState: Order Confirmed
7. DoState: Decide upon further action
   Branches:
     - Step: 8
       Description: Retry with another supplier
     - Step: 9
       Description: Cancel order
8. GotoStep: 2  # Retry order loop
9. SendState: Send Order Cancellation Request to E-commerce Website
   To: E-commerce Website
   Msg: Order Cancellation Request
10. EndState: Order Cancelled

### Explanation:

- The `Customer` starts by deciding to place an order and sends an `Order` to the E-commerce Website.
- The `Customer` then receives a response from the E-commerce Website, which could be either a `Payment Request` or an `Order Declination Message`.
- If a `Payment Request` is received, the `Customer` sends a `Payment` to the Payment Gateway.
- The `Customer` then receives a reply from the Payment Gateway, which could be either a `Payment Confirmation` or a `Payment Declination`.
- If a `Payment Confirmation` is received, the process ends with the order confirmed.
- If a `Payment Declination` is received, the `Customer` decides upon further action, which could be to retry with another supplier or to cancel the order.
- If the `Customer` chooses to retry with another supplier, the flow returns to the original `SendState` for sending the order.
- If the `Customer` chooses to cancel the order, they send an `Order Cancellation Request` to the E-commerce Website and the process ends.
"""

###########Mistral Response Obtained with DoState Description/Label
sbd_response_text4 = """
### SBD for Customer:

1. StartState: Decide to place an order
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website
   To: E-commerce Website
   Msg: Order

3. ReceiveState: Receive Reply from E-commerce Website
   Choices:
     - From: E-commerce Website
       Msg: Payment Request
       Next: 4
     - From: E-commerce Website
       Msg: Order Declination Message
       Next: 5

4. SendState: Send Payment to Payment Gateway
   To: Payment Gateway
   Msg: Payment

5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
     - From: Payment Gateway
       Msg: Payment Confirmation
       Next: 6
     - From: Payment Gateway
       Msg: Payment Declination
       Next: 7

6. EndState: Order Confirmed

7. DoState: Decide upon further action
   Branches:
     - Step: 8
       Description: Cancel order
     - Step: 10
       Description: Choose another supplier

8. SendState: Send Order Cancellation Request to E-commerce Website
   To: E-commerce Website
   Msg: Order Cancellation Request

9. EndState: Order Cancelled

10. GotoStep: 2 # Retry order loop

### Explanation:

1. **StartState: Decide to place an order**
   - The customer decides to place an order on the e-commerce website.

2. **SendState: Send Order to E-commerce Website**
   - The customer sends an order to the e-commerce website.

3. **ReceiveState: Receive Reply from E-commerce Website**
   - The customer receives a reply from the e-commerce website.
   - If the item is in stock, the customer receives a `Payment Request`.
   - If the item is not in stock, the customer receives an `Order Declination Message`.

4. **SendState: Send Payment to Payment Gateway**
   - The customer sends a payment to the payment gateway.

5. **ReceiveState: Receive Reply from Payment Gateway**
   - The customer receives a reply from the payment gateway.
   - If the payment is confirmed, the customer receives a `Payment Confirmation`.
   - If the payment is declined, the customer receives a `Payment Declination`.

6. **EndState: Order Confirmed**
   - The order is confirmed and the process ends.

7. **DoState: Decide upon further action**
   - The customer decides upon further action after receiving a `Payment Declination`.

8. **SendState: Send Order Cancellation Request to E-commerce Website**
   - The customer sends an order cancellation request to the e-commerce website.

9. **EndState: Order Cancelled**
   - The order is cancelled and the process ends.

10. **GotoStep: 2 # Retry order loop**
    - The customer decides to choose another supplier and the process loops back to the original order step.
"""

combined_response = sid_response_text.strip() + "\n" + sbd_response_text4.strip()

def parse_sid(sid_text):
    pattern = re.compile(r"\d+\.\s*(.+?)\s*->\s*(.+?):\s*(.+)")
    sid = []
    for line in sid_text.splitlines():
        match = pattern.match(line.strip())
        if match:
            sender = match.group(1).strip()
            receiver = match.group(2).strip()
            message = match.group(3).strip()
            sid.append((f"{sender} -> {receiver}", message))
    return sid

def extract_sbd_section(full_text):
    lines = full_text.splitlines()
    sbd_start = None
    for i, line in enumerate(lines):
        if "### SBD" in line:
            sbd_start = i + 1  # start after this line
            break
    if sbd_start is None:
        print("No SBD section found!")
        return ""

    # Extract until explanation or EOF
    sbd_lines = []
    for line in lines[sbd_start:]:
        if line.strip().startswith("### Explanation"):
            break
        sbd_lines.append(line)

    return "\n".join(sbd_lines).strip()
    

def parse_sbd(sbd_text):
    print("=== SBD Text to parse ===")
    print(sbd_text)
    print("=========================")
    lines = sbd_text.splitlines()

    sbd = {}
    current_system = None
    states = {}
    current_state = None
    current_state_num = None
    choices = []
    branches = []
    parsing_choices = False
    parsing_branches = False

    actor_header_re = re.compile(r"^\s*####\s*(\w+):")
    state_header_re = re.compile(r"(\d+)\.\s+\*{0,2}(\w+)\*{0,2}:\s*(.*)")
    goto_re = re.compile(r"(\d+)\.\s+\*{0,2}GotoStep\*{0,2}:\s*(\d+)")

    for idx, line in enumerate(lines):
        line = line.rstrip()
        if not line:
            continue

        # Detect actor header
        m = actor_header_re.match(line)
        if m:
            # Debug
            print(f"Detected actor: {m.group(1)}")

            # Save previous actor's states
            if current_system is not None:
                sbd[current_system] = list(states.values())

            current_system = m.group(1)
            states = {}
            current_state = None
            current_state_num = None
            choices = []
            branches = []
            parsing_choices = False
            parsing_branches = False
            continue

        if current_system is None:
            # Haven't detected an actor header yet
            current_system = "Default"
            states = {}
           

        m = goto_re.match(line)
        if m:
            current_state_num = int(m.group(1))
            next_step = int(m.group(2))
            current_state = {
                "type": "GotoStep",
                "description": f"Goto step {next_step}",
                "num": current_state_num,
                "GotoStep": next_step
            }
            states[current_state_num] = current_state
            continue

        m = state_header_re.match(line)
        if m:
            # Save previous state's choices or branches if any
            if current_state is not None:
                if choices:
                    current_state["Choices"] = choices
                    choices = []
                if branches:
                    current_state["Branches"] = branches
                    branches = []

            current_state_num = int(m.group(1))
            state_type = m.group(2)
            description = m.group(3).strip()
            current_state = {
                "type": state_type,
                "description": description,
                "num": current_state_num
            }
            
            # Check next line for edge description, but only if it's StartState or DoState with no branches
            
            if state_type in ["StartState", "DoState"]:
                # Only look ahead if it's not followed by "Branches:"
                next_lines = lines[idx+1:idx+3]  # Look ahead one or two lines safely
                has_branches_soon = any(l.strip() == "Branches:" for l in next_lines)
                if not has_branches_soon:
                    for next_line in next_lines:
                        next_line = next_line.strip()
                        if next_line.startswith("Description:"):
                            current_state["OutgoingLabel"] = next_line.split(":", 1)[1].strip()
                            break

            states[current_state_num] = current_state
            parsing_choices = False
            parsing_branches = False
            continue

        if current_state is None:
            continue

        # Handle SendState To: and Msg:
        if current_state["type"] == "SendState":
            if line.strip().startswith("To:"):
                current_state["To"] = line.split(":",1)[1].strip()
                continue
            if line.strip().startswith("Msg:"):
                current_state["Msg"] = line.split(":",1)[1].strip()
                continue

        # Detect Choices and Branches
        if line.strip() == "Choices:":
            parsing_choices = True
            parsing_branches = False
            choices = []
            continue
        elif line.strip() == "Branches:":
            parsing_branches = True
            parsing_choices = False
            branches = []
            continue

        if parsing_choices:
            if line.strip().startswith("- From:"):
                choices.append({"From": line.split(":",1)[1].strip()})
            elif line.strip().startswith("Msg:") and choices:
                choices[-1]["Msg"] = line.split(":",1)[1].strip()
            elif line.strip().startswith("Next:") and choices:
                choices[-1]["Next"] = int(line.split(":",1)[1].strip())
            continue

        if parsing_branches:
            if line.strip().startswith("- Step:"):
                branches.append({"Step": int(line.split(":",1)[1].strip())})
            elif line.strip().startswith("Description:") and branches:
                branches[-1]["Description"] = line.split(":",1)[1].strip()
            continue

    # Save last state's choices or branches
    if current_state is not None:
        if choices:
            current_state["Choices"] = choices
        if branches:
            current_state["Branches"] = branches

    # Save last actor's states
    if current_system is not None:
        sbd[current_system] = list(states.values())

    return sbd

 
    
sbd_textt = extract_sbd_section(combined_response)

parsed_sbd = parse_sbd(sbd_textt)
print("Parsed SBD::", parsed_sbd)


sid_list = parse_sid(sid_response_text.strip())
process = {
    "SID": sid_list
}


print("SID Text:\n", sid_response_text.strip())

=== SBD Text to parse ===
1. StartState: Decide to place an order
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website
   To: E-commerce Website
   Msg: Order

3. ReceiveState: Receive Reply from E-commerce Website
   Choices:
     - From: E-commerce Website
       Msg: Payment Request
       Next: 4
     - From: E-commerce Website
       Msg: Order Declination Message
       Next: 5

4. SendState: Send Payment to Payment Gateway
   To: Payment Gateway
   Msg: Payment

5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
     - From: Payment Gateway
       Msg: Payment Confirmation
       Next: 6
     - From: Payment Gateway
       Msg: Payment Declination
       Next: 7

6. EndState: Order Confirmed

7. DoState: Decide upon further action
   Branches:
     - Step: 8
       Description: Cancel order
     - Step: 10
       Description: Choose another supplier

8. SendState: Send Order Cancellation Request to E-commerce Website
   To

In [4]:
##############SID Graph##########################
import os
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

from graphviz import Digraph
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")  # Make sure this is installed

# Define canonical entities
canonical_entities = [
    "Customer",
    "E-commerce Website"    
]

# Convert to spaCy Docs for similarity comparison
canonical_docs = {ent: nlp(ent) for ent in canonical_entities}

def normalize_entity(entity):
    entity_doc = nlp(entity)
    best_match = None
    best_score = 0.75  # Similarity threshold (tune as needed)

    for canon_name, canon_doc in canonical_docs.items():
        score = entity_doc.similarity(canon_doc)
        if score > best_score:
            best_match = canon_name
            best_score = score

    return best_match if best_match else entity.strip()

def draw_sid_graph(sid_list, output_path="sid_graph"):
    dot = Digraph(comment="SID - Sequence Interaction Diagram")
    dot.attr(rankdir='LR',splines='polyline')  # Left to right direction
    
    # Deduplicate using a set
    normalized_edges = set()

    for interaction, action in sid_list:
        source, target = [normalize_entity(s.strip()) for s in interaction.split("->")]
        key = (source, target, action.strip())

        if key in normalized_edges:
            print(f"Skipping duplicate: {source} -> {target} [{action}]")
            continue

        normalized_edges.add(key)
        dot.node(source, source, shape="box", style="filled", fillcolor="lightblue")
        dot.node(target, target, shape="box", style="filled", fillcolor="lightgreen")
        dot.edge(source, target, label=action)

    # Render graph to file
    dot.render(output_path, format='png', cleanup=True)
    print(f"SID graph saved to: {output_path}.png")
    
#Log the SID interactions being drawn
print("Drawing SID arrows:")
for interaction, action in process["SID"]:
    print(f"{interaction} [{action}]")

# Render SID graph
draw_sid_graph(
    process["SID"],
    output_path="/home/s/smanan/sid_graph"
)

Drawing SID arrows:
Customer -> E-commerce Website [Order]
E-commerce Website -> Inventory System [Inventory Check Request]
Inventory System -> E-commerce Website [Inventory Check Response]
E-commerce Website -> Customer [Payment Request]
Customer -> Payment Gateway [Payment]
Payment Gateway -> E-commerce Website [Payment Confirmation]
E-commerce Website -> Order Confirmation System [Order Confirmation Request]
Order Confirmation System -> Customer [Order Confirmation Email]
Inventory System -> E-commerce Website [Out of Stock Notification]
E-commerce Website -> Customer [Order Declination Message]
Customer -> E-commerce Website [Order Cancellation Request]
Customer -> Another E-commerce Supplier [Order]
Skipping duplicate: Customer -> E-commerce Website [Order]
SID graph saved to: /home/s/smanan/sid_graph-June20.png


In [9]:
import os

# Add the environment bin directory to PATH
os.environ["PATH"] += os.pathsep + "/home/s/smanan/.conda/envs/myenv/bin"

# Optional: test that dot is now found
!which dot
!dot -V

/home/s/smanan/.conda/envs/myenv/bin/dot
dot - graphviz version 12.2.1 (20250203.1708)


In [44]:
#######################################SBD Graph#####################################
import re
from graphviz import Digraph

dot = Digraph("SBD", format='pdf')
dot.attr(rankdir='LR')

state_styles = {
    'StartState': {"shape": "rectangle", "color": "yellow"},
    'EndState':   {"shape": "rectangle", "color": "yellow"},
    'SendState':  {"shape": "rectangle", "color": "green"},
    'ReceiveState': {"shape": "rectangle", "color": "pink"},
    'DoState':    {"shape": "rectangle", "color": "yellow"},
    'Unknown':    {"shape": "rectangle", "color": "lightgrey"},
}


steps = {}
lines = sbd_textt.strip().splitlines()
current_step = None


i = 0
while i < len(lines):
    line = lines[i].strip()

   
    match = re.match(r'^(\d+)\.\s*\*{0,2}(\w+)\*{0,2}:\s*(.*)', line)
    if match:
        num, state_type, label = match.groups()
        current_step = int(num)

        if state_type == "GotoStep":
            target_step = int(label.strip().split()[0])
            steps[current_step] = {
                "type": "GotoStep",
                "label": "",
                "choices": [],
                "branches": [],
                "goto": target_step
            }
        else:
            steps[current_step] = {
                "type": state_type,
                "label": label,
                "choices": [],
                "branches": [],
                "goto": None
            }
            # If it's a SendState, look ahead to capture 'To' and 'Msg'
            if state_type == "SendState":
                if i + 1 < len(lines) and lines[i + 1].strip().startswith("To:"):
                    steps[current_step]["To"] = lines[i + 1].strip().split(":", 1)[1].strip()
                    i += 1
                if i + 1 < len(lines) and lines[i + 1].strip().startswith("Msg:"):
                    steps[current_step]["Msg"] = lines[i + 1].strip().split(":", 1)[1].strip()
                    i += 1

            # For StartState and DoState, check if next line(s) contain OutgoingLabel
            if state_type in {"StartState", "DoState"}:
                # Look ahead for OutgoingLabel line(s)
                if i + 1 < len(lines) and lines[i + 1].strip().startswith("OutgoingLabel:"):
                    steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                    i += 1
                # If no OutgoingLabel, look for Description and use that as OutgoingLabel
                elif i + 1 < len(lines) and lines[i + 1].strip().startswith("Description:"):
                    steps[current_step]["OutgoingLabel"] = lines[i + 1].strip().split(":", 1)[1].strip()
                    i += 1
            i += 1
            continue

    # Parse ReceiveState choices
    elif line.startswith('- From:') and current_step is not None:
        from_actor = line.replace('- From:', '').strip()
        msg = ""
        next_step = None

        if i + 1 < len(lines) and lines[i + 1].strip().startswith('Msg:'):
            msg = lines[i + 1].strip().split(':', 1)[1].strip()
            i += 1
        if i + 1 < len(lines) and lines[i + 1].strip().startswith('Next:'):
            next_step = int(lines[i + 1].strip().split(':', 1)[1].strip())
            i += 1

        steps[current_step]["choices"].append({
            "from": from_actor,
            "msg": msg,
            "next": next_step
        })

    # Parse DoState branches
    elif line.startswith('- Step:') and current_step is not None:
        step = int(line.split(':')[1].strip())
        desc = ""
        if i + 1 < len(lines) and lines[i + 1].strip().startswith('Description:'):
            desc = lines[i + 1].strip().split(':', 1)[1].strip()
            i += 1

        steps[current_step]["branches"].append({
            "step": step,
            "desc": desc
        })

    # Goto inside other step types
    elif 'GotoStep' in line and current_step is not None:
        
        match = re.match(r'\*{0,2}GotoStep\*{0,2}:\s*(\d+)', line)
        if match:
            steps[current_step]["goto"] = int(match.group(1))

    i += 1

# Add dummy nodes for unresolved goto targets
goto_targets = {info["goto"] for info in steps.values() if info["goto"] is not None}
for tgt in goto_targets:
    if tgt not in steps:
        steps[tgt] = {
            "type": "Unknown",
            "label": f"Step {tgt} \\nundefined",
            "choices": [],
            "branches": [],
            "goto": None
        }

print("Parsed steps:")
for k, v in steps.items():
    print(k, v)
    
# === Draw state nodes ===
for num, info in steps.items():
    if info["type"] == "GotoStep":
        continue  # skip drawing node
    
    if info["type"] == "Unknown" and info["label"].endswith("undefined"):
        continue

    style = state_styles.get(info["type"], {"shape": "rectangle", "color": "lightgrey"})
    dot.node(str(num), label=info["label"], shape=style["shape"], style="filled", fillcolor=style["color"])

# === Draw transitions ===
all_step_nums = sorted(steps.keys())

for idx, num in enumerate(all_step_nums):
    info = steps[num]

    if info["type"] == "GotoStep":
        continue  # Only draw the goto edge, skip all else

    if info["type"] == "EndState":
        continue

    if info["type"] == "ReceiveState":
        for choice in info["choices"]:
            label = f"Frm={choice['from']}\\nMsg={choice['msg']}"
            dot.edge(str(num), str(choice["next"]), label=label)
            
    elif info["type"] == "SendState":
        label_parts = []
        if "To" in info:
            label_parts.append(f"To={info['To']}")
        if "Msg" in info:
            label_parts.append(f"Msg={info['Msg']}")
        label = "\\n".join(label_parts)

        next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
        if next_step:
            dot.edge(str(num), str(next_step), label=label)
                 
    elif info["type"] == "StartState":
        next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
        if next_step:
            label = info.get("OutgoingLabel", info["label"])
            dot.edge(str(num), str(next_step), label=label)

    elif info["type"] == "DoState":
        if info["branches"]:
            for branch in info["branches"]:
                target_step = branch["step"]
                target_info = steps.get(target_step)
                if target_info and target_info["type"] == "GotoStep" and target_info.get("goto"):
                    real_target = target_info["goto"]
                    dot.edge(str(num), str(real_target), label=branch["desc"])
                else:
                    dot.edge(str(num), str(target_step), label=branch["desc"])
        else:
            # No branches — fall back to sequential next step using description as label
            next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
            if next_step:
                label = info.get("OutgoingLabel", info["label"])
                dot.edge(str(num), str(next_step), label=label)


    if info["goto"]:
        dot.edge(str(num), str(info["goto"]))
        continue

    next_step = all_step_nums[idx + 1] if idx + 1 < len(all_step_nums) else None
   
    
    has_outgoing = (
    bool(info["choices"]) or
    bool(info["branches"]) or
    bool(info["goto"]) or
    info["type"] in {"SendState", "DoState", "StartState"}
    )

    if next_step and not has_outgoing:
        dot.edge(str(num), str(next_step))

# Save and render

dot.render("/home/s/smanan/sbd_visualization", cleanup=True)

Parsed steps:
1 {'type': 'StartState', 'label': 'Decide to place an order', 'choices': [], 'branches': [], 'goto': None, 'OutgoingLabel': 'Customer initiates order process'}
2 {'type': 'SendState', 'label': 'Send Order to E-commerce Website', 'choices': [], 'branches': [], 'goto': None, 'To': 'E-commerce Website', 'Msg': 'Order'}
3 {'type': 'ReceiveState', 'label': 'Receive Reply from E-commerce Website', 'choices': [{'from': 'E-commerce Website', 'msg': 'Payment Request', 'next': 4}, {'from': 'E-commerce Website', 'msg': 'Order Declination Message', 'next': 5}], 'branches': [], 'goto': None}
4 {'type': 'SendState', 'label': 'Send Payment to Payment Gateway', 'choices': [], 'branches': [], 'goto': None, 'To': 'Payment Gateway', 'Msg': 'Payment'}
5 {'type': 'ReceiveState', 'label': 'Receive Reply from Payment Gateway', 'choices': [{'from': 'Payment Gateway', 'msg': 'Payment Confirmation', 'next': 6}, {'from': 'Payment Gateway', 'msg': 'Payment Declination', 'next': 7}], 'branches': [], 

'/home/s/smanan/sbd_visualization_June19_5.pdf'

In [29]:
from openai import OpenAI #############Prompt 2 SBD Version 3 With DoState and StartState Labels

# Setup
api_key = ""
base_url = "https://gpt.uni-muenster.de/v1"
model = "mistral-small"  # model used

client = OpenAI(api_key=api_key, base_url=base_url)


# Define scenario
scenario = """
A customer places an order on an e-commerce website. 
The system checks inventory. 
If the item is in stock, customer is informed to pay for the product.
Customer pays for the order.
The payment gateway confirms the payment transaction. 
Finally, the system sends order confirmation email to the customer.
But if the item is not in stock, order is declined.
Customer is sent order declined message.
Customer then decides upon further action either the customer decides to cancel order and it ends
or customer chooses another e-commerce supplier.
"""


# Create the prompt
prompt = f"""
You are an expert in Subject-Oriented Business Process Modeling using the Parallel Activity Specification Schema (PASS).

Here is the original scenario:

\"\"\"{scenario}\"\"\"

Based on the Subject Interaction Diagram (SID) below, generate the Subject Behavior Diagram (SBD) for the **Customer** subject.

### SID:
1. Customer -> E-commerce Website: Order
2. E-commerce Website -> Inventory System: Inventory Check Request
3. Inventory System -> E-commerce Website: Inventory Check Response
4. E-commerce Website -> Customer: Payment Request
5. Customer -> Payment Gateway: Payment
6. Payment Gateway -> E-commerce Website: Payment Confirmation
7. E-commerce Website -> Order Confirmation System: Order Confirmation Request
8. Order Confirmation System -> Customer: Order Confirmation Email
9. Inventory System -> E-commerce Website: Out of Stock Notification
10. E-commerce Website -> Customer: Order Declination Message
11. Customer -> E-commerce Website: Order Cancellation Request
12. Customer -> Another E-commerce Supplier: Order


### Rules for the SBD:

- For each subject, describe their **internal behavior** as a sequence of states.
- Use the following state types and label each step explicitly:
  - **StartState**: the beginning of the behavior
  - **ReceiveState**: receiving a message
  - **SendState**: sending a message
      - For every **SendState**, include the following two fields on the next lines:
      - `To:` — the name of the recipient subject
      - `Msg:` — the message being sent, written as a **noun or noun phrase** (e.g., `Order`, `Payment`, `Order Cancellation Request`)

      Example:
      2. SendState: Send Order to E-commerce Website  
         To: E-commerce Website  
         Msg: Order

- **DoState**: internal processing or logic
  - **EndState**: conclusion of the subject's behavior
  
- Ensure each interaction in the SID maps to a SendState and ReceiveState in the corresponding SBDs.
- For every **ReceiveState**, also include the `From:` (sender subject) and `Msg:` (the received noun message) on the next line below it.
  - The `Msg:` must be a **noun or noun phrase only**, matching the noun form used in the SID (e.g., "Order", "Appointment Confirmation").  
  - Do **not** use verbs, verb phrases, or action words (e.g., "send payment", "check availability").

  
### Control Flow & Branching Logic (Important):
- A `DoState` can act as a **decision point**.
- If a state has multiple `Branches:`, they represent **mutually exclusive outcomes**, not parallel or sequential steps.
- Use `Branches:` to list possible next steps in decision states.
- To represent non-linear flow, use `GotoStep:` to explicitly jump back or forward to a specific step.
- Do **not** assume that steps are sequential by default—follow the above rules.
- End a branch with `EndState` or no further transitions to mark it as terminal.

-Use GotoStep: to define non-linear transitions (e.g., retry loops or skipping steps).
-A ReceiveState can lead to multiple different outcomes based on the received message.
   Use a Choices: block to specify alternative messages and the corresponding next steps.
   Each choice must include:
     From: — the sender subject
     Msg: — the received message (noun phrase)
     Next: — the step that follows if that message is received
     
Clarifications for ReceiveState and Control Flow:

- A `ReceiveState` with `Choices:` should **not** continue into the next numeric step by default.
- The `Next:` field under each `Choice:` is the **only** path to follow.
- Ensure each `Next:` maps to a valid step with correct logic. Do **not** assume the next numeric step follows sequentially.
- If a message like `Order Declination Message` is received, it should go **directly to the correct decision step** (e.g., Decide upon further action), not to a step meant for confirmations (e.g., Step 5).
- Use `GotoStep:` to connect flows clearly after decisions or retries.
- Branches in a `DoState` represent **mutually exclusive decisions** and should not be followed sequentially.

### Additional Structural Rules for Valid ReceiveStates:

- Each `ReceiveState` should group only messages that are logically possible **at that specific point in the behavior**. Do not mix messages from different process phases or inconsistent branches.
  
- After a subject has performed a critical action (e.g., sent a payment, confirmed a booking), they should not receive a message that contradicts or invalidates that action (e.g., an order declination after payment).  
  - Such contradictory messages should be handled **before** that action was taken — not afterward.

- A `ReceiveState` must not mix messages from **different sender subjects** that belong to **separate branches of logic**. For example:
  - If one message implies success from one subject (e.g., confirmation email from System A)
  - And another implies failure from another subject (e.g., declination from System B)
  - These should not be in the same `Choices:` block. They should belong to **different ReceiveStates**, based on earlier branching.

- Always ensure that once a decision or branching has occurred, only the resulting possible messages **from that path** are expected in the next `ReceiveState`.

- If a message like `Order Declination` has already been handled in a previous `ReceiveState` as a decision point, it should **not appear again** in later states.

### Additional Rule: Include Description for StartState and DoState

- For every **StartState** and **DoState**, provide a `Description:` line immediately below the step.
- This line must concisely describe what causes the state to exit or what condition/decision leads to the next transition.
- This helps label the outgoing edge in visual diagrams.

**Examples**:

1. StartState: Decide to place an order  
   Description: Customer initiates order process

2.DoState: Pick Order
  Description: Order picked

### Structural and Control Flow Guidance:

- Do **not** create redundant ReceiveState steps. If a message is received at multiple points (e.g., "Order Declination Message"), handle all logic in the **same ReceiveState step**, using `Choices:` and `Next:`.
  - Do **not** repeat the same ReceiveState (e.g., ReceiveState: Receive Reply from E-commerce Website) if some previous Step already handles the same messages.
- If a behavior is a repeat (e.g., re-sending the order), do **not** create a new `SendState`.  
  - Use `GotoStep:` to return to the original step (e.g., `GotoStep: 2`), rather than creating a redundant step like "SendState: Send Order for Another Supplier to E-commerce Website".

- `GotoStep:` should be used for loops (e.g., retrying with another supplier). Avoid creating a new SendState when the action already exists in a previous step.

- When listing `Branches:`, each branch should either:
  - Lead to an `EndState`, or
  - Lead to another state (e.g., `DoState`, `SendState`) and eventually terminate or loop.
  
###IMPORTANT:  
-If the ReceiveState choice is regarding Declination or rejection then next state number should belong to Decide upon further action. 
  
### Additional Rules for Correct EndStates and Branching:

- When using a `DoState` with `Branches:`, each branch must:
  - Lead clearly to an `EndState`, OR
  - Use a `GotoStep:` to continue the flow elsewhere.

- If a branch leads to an intermediate action (like sending a cancellation request), then that action **must be followed explicitly by an `EndState`** unless the flow continues elsewhere.

- Do **not** include orphan `EndState` steps that are never referenced via `Next:` or `GotoStep:`. Every `EndState` must be clearly reachable in the flow.

- You may combine a final action and end state if there is no further processing (e.g., "Send Cancellation Request and terminate" can be modeled in a single `SendState` followed immediately by `EndState`).

- When retrying an earlier action (like sending a new order), use `GotoStep:` to return to that earlier state. Do not re-declare the same SendState.

### Final State Rule:
- A single step cannot combine a `SendState` and `EndState`.  
- If a final action like sending a cancellation request ends the process:
  - Use two steps:
    - One `SendState`
    - One `EndState` directly after
- Example:
  9. SendState: Send Cancellation Request to E-commerce Website  
  10. EndState: Order Cancelled

- Then reference Step 10 explicitly from any branch or choice that leads there.


### Post-Payment Behavior:

- After sending a payment, the Customer must wait for a response using a `ReceiveState`.
- This `ReceiveState` should include:
  - A `Payment Confirmation` message that proceeds to an `EndState` or confirmation step.
  - A `Payment Declination` message that leads to a decision point (`DoState`).

### Avoid Ambiguous Transitions:

- Do not place a `DoState` directly after sending a payment.
- A `DoState` is only appropriate after:
  - A declination has been received.
  - The customer must choose between canceling or retrying.

### Retry Logic:

- If a retry loop is needed, use:
  - `GotoStep:` to return to the appropriate order initiation step.
- Ensure all retry paths explicitly flow to `GotoStep`, not a dangling step.

### IMPORTANT Branch Semantics and Retry Detection Rules:
-If a Branch has a description that includes any of the following retry-like phrases:

 -choose another [supplier|hospital|partner]
 -try again
 -retry
 -resend
 -submit again
 -place another order

Then that branch must lead to a GotoStep pointing to the step where the original send action occurred (e.g., sending the order or request).
Do not create a new SendState — reuse the earlier one via GotoStep:.
This enables looping retry logic without duplication.
These kinds of retry branches should never point to an EndState.
If the intent of the branch is to cancel or stop, use a SendState for final communication and then an EndState.

Example:
5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel order
     - Step: 8
       Description: Choose another supplier
6. SendState: Send Order Cancellation Request
7. EndState: Order Cancelled
8. GotoStep: 2  # Retry order loop

### Step Types:

- A step must use **only one state type**: either `SendState`, `ReceiveState`, `DoState`, or `EndState`.
- Do not combine `SendState` and `EndState` in one step.


- If a retry is needed (e.g., choosing another supplier), use a `GotoStep:` to return to the appropriate earlier step (usually where the order is sent).
- Do **not** define a `GotoStep:` without referencing it in any branch.
- Every GotoStep must be **actually used in a `Branches:` section** of a `DoState`.

- Every `GotoStep:` must be referenced explicitly in a decision branch or as a next step in the control flow.
- Do not create `GotoStep:` steps that are not reachable from any previous state.
- When defining `Branches:` in a `DoState`, ensure that each branch points to a valid next step, including `GotoStep:` steps if needed.
- For example, if you define a retry loop as `GotoStep: 2`, ensure the decision branch description points to that step number.
- Avoid orphan steps that are unreachable from the flow. Like this below, 8. GotoStep: 2  # Retry appointment loop is referenced explicitly
in Step 5 - Step: 8
       Description: Choose another hospital and try again :

5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel appointment request 
     - Step: 8
       Description: Choose another hospital and try again  
6. DoState: Cancel appointment request
7  EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

  
### Use the following format (example based on a healthcare scenario):

### Example Scenario:
A patient books an appointment using a healthcare app.  
The app sends the appointment request to the hospital system.  
The hospital system checks the doctor’s availability.  
If the doctor is available, it confirms the appointment.  
The app then sends a confirmation message to the patient.
If the doctor is not available, hospital system declines the appointment.
The patient is sent decline message.
The patient then decides upon further action either the patient cancels request and all ends
or the patient chooses another hospital.

### SBD:

#### Patient:
1. StartState: Decide to make appointment
   Description: Patient initiates appointment process
2. SendState: Send Appointment Request to Healthcare App  
   -To:  Healthcare App
   -Msg: Appointment Request
3. ReceiveState: Receive Reply from Healthcare App
   Choices:
   -From: Healthcare App  
    Msg: Appointment Confirmation   
    Next: 4
   -From: Healthcare App
    Msg: Appointment Declined
    Next: 5 
   
4. EndState: Appointment booked
5. DoState: Decide upon further action
   Branches:
     - Step: 6  
       Description: Cancel appointment request 
     - Step: 8
       Description: Choose another hospital and try again  
6. DoState: Cancel appointment request
   Description: Cancel
7  EndState: Appointment request cancelled (end of this path)
8. GotoStep: 2  # Retry appointment loop

Note: Step 5 must be the exclusive next step after receiving an "Appointment Declined" message. Similarly, in the e-commerce context, if the customer receives an "Order Declination Message", it must always go to this decision point, never to a step meant for payment.

## IMPORTANT RULE:
If the `Customer` receives an `Declination Message`, it must transition to a `DoState` titled `Decide upon further action`. This state must contain:
- A branch to cancel the order (leading to a SendState, then EndState)
- A branch to choose another supplier (leading to `GotoStep: 2`)

Under no condition should the `Order Declination Message` lead directly to the payment step or any unrelated state.

##IMPORTANT:
Ensure that state numbering reflects logical flow.
For example:

If a payment is sent in Step 4, then receiving the payment response must occur immediately after (e.g., in Step 5 or Step 5.x), not at the end.

Avoid placing follow-up states (like payment confirmation) far from their corresponding initiator steps.

Only GotoStep or loops should cause jumps in the flow.

Try to keep closely related actions (send/receive pairs, decision/action results) in proximity and numbered accordingly.

Now, generate the SBD for Customer only.

Important Correction:
If the customer chooses to try another supplier, use `GotoStep: 2` (Send Order....) rather than `GotoStep: 3` (Receive Response). The retry loop must start from the original `SendState`, not from the `ReceiveState`.
"""

# Make the API call
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=model,
    temperature=0.5
)

# Output
print(completion.choices[0].message.content)

### Subject Behavior Diagram (SBD) for Customer:

1. StartState: Decide to place an order
   Description: Customer initiates order process

2. SendState: Send Order to E-commerce Website
   To: E-commerce Website
   Msg: Order

3. ReceiveState: Receive Response from E-commerce Website
   Choices:
     - From: E-commerce Website
       Msg: Payment Request
       Next: 4
     - From: E-commerce Website
       Msg: Order Declination Message
       Next: 5

4. SendState: Send Payment to Payment Gateway
   To: Payment Gateway
   Msg: Payment

5. ReceiveState: Receive Reply from Payment Gateway
   Choices:
     - From: Payment Gateway
       Msg: Payment Confirmation
       Next: 6
     - From: Payment Gateway
       Msg: Payment Declination
       Next: 7

6. ReceiveState: Receive Order Confirmation Email
   Choices:
     - From: Order Confirmation System
       Msg: Order Confirmation Email
       Next: 8

7. DoState: Decide upon further action
   Branches:
     - Step: 9
       Descripti